In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

False

In [3]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [4]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [5]:
with open("tagged_description.txt", "w", encoding="utf-8") as f:
    for idx, description in books["tagged_description"].items():
        # Handle potential NaN values
        if pd.notna(description):
            f.write(str(description) + "\n")
        else:
            f.write(f"No description available for book {idx}\n")

In [6]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 479, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 284, which is longer than the specified 0
Created a chunk of size 193, which is longer than the specified 0
Created a chunk of size 877, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books = Chroma.from_documents(documents,
                                 embedding=embeddings,)

In [8]:
query = ("A book about irish history")
docs = db_books.similarity_search(query, k = 1)

def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)


In [9]:
retrieve_semantic_recommendations(query)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
9,9780006483014,0006483011,The Once and Future King,Terence Hanbury White,Arthurian romances,http://books.google.com/books/content?id=Jx6Bv...,An omnibus volume of the author's complete sto...,1996.0,4.04,823.0,2805.0,The Once and Future King,9780006483014 An omnibus volume of the author'...
17,9780006496922,000649692X,Glamorous Powers,Susan Howatch,Clergy,http://books.google.com/books/content?id=_bhPY...,Reissue of the author's most famous and well-l...,1996.0,4.20,512.0,1441.0,Glamorous Powers,9780006496922 Reissue of the author's most fam...
254,9780060841867,0060841869,The Curtain,Milan Kundera,Literary Collections,http://books.google.com/books/content?id=JCDXP...,Traces the author's personal view of the histo...,2006.0,3.96,176.0,1351.0,The Curtain: An Essay in Seven Parts,9780060841867 Traces the author's personal vie...
331,9780060975944,0060975946,The Fourteen Sisters of Emilio Montez O'Brien,Oscar Hijuelos,Fiction,http://books.google.com/books/content?id=B2jEu...,A portrait of America as seen through the eyes...,1996.0,3.69,496.0,457.0,The Fourteen Sisters of Emilio Montez O'Brien,9780060975944 A portrait of America as seen th...
400,9780062700254,0062700251,Bulfinch's Mythology,Richard P. Martin,Reference,http://books.google.com/books/content?id=eev4u...,A beautiful gift edition of Thomas Bulfinch's ...,1991.0,4.10,768.0,64.0,"Bulfinch's Mythology: The Age of the Fable, Th...",9780062700254 A beautiful gift edition of Thom...
415,9780064406307,006440630X,The Midwife's Apprentice (rpkg),Karen Cushman,Juvenile Fiction,http://books.google.com/books/content?id=Bhm76...,"'Like Cushman's 1995 Newbery Honor Book, Cathe...",1996.0,3.72,128.0,35319.0,The Midwife's Apprentice (rpkg),9780064406307 'Like Cushman's 1995 Newbery Hon...
478,9780099422341,0099422344,Yeats is Dead!,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,Yeats is Dead!: A Novel by Fifteen Irish Writers,"9780099422341 In aid of Amnesty International,..."
527,9780099498582,0099498588,Dublin 4,Maeve Binchy,Domestic fiction,http://books.google.com/books/content?id=yPgE7...,A vintage collection of short stories set in t...,2006.0,3.64,256.0,2089.0,Dublin 4,9780099498582 A vintage collection of short st...
564,9780140137002,0140137009,Montaillou,Emmanuel Le Roy Ladurie,Albigenses,http://books.google.com/books/content?id=gBZ7Q...,An enthralling account of day-to-day life in a...,1990.0,3.97,400.0,71.0,Montaillou,9780140137002 An enthralling account of day-to...
601,9780140247749,0140247742,Dubliners,James Joyce;Robert Scholes;A. Walton Litz,Fiction,http://books.google.com/books/content?id=yOR4Q...,"In these masterful stories, steeped in realism...",1996.0,3.85,492.0,320.0,Dubliners,"9780140247749 In these masterful stories, stee..."
